## Blockchain

In [1]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd


import seaborn as sns

%load_ext autoreload
%autoreload 2

In [ ]:
def fetch_lyrics(artist, title):    
    url = "https://lyrics.lewagon.ai/search?"
    params_dict = {'artist':artist,'title':title}
    response = requests.get(url, params = params_dict)
    if response.ok:
        return response.json()['lyrics']  ## checking if we have lyrics in the DICT
    return ''

In [ ]:
%%time
for index, row in tracks_df.head(5).iterrows(): # iterrows to iterate attraverso DF
    print(f"Fetching lyrics for {row['artists']} - {row['name']}")
    lyrics = fetch_lyrics_2(row['artists'], row['name'])
    tracks_df.loc[index, 'lyrics'] = lyrics

In [46]:
import datetime


In [ ]:
pd.read_scv('../../Data_project')

In [20]:
import pandas as pd
import os
data_path = '../../Data_project'
df = pd.read_csv(os.path.join(data_path, 'nvt.csv'), index_col=0 ,parse_dates=True)

In [37]:
df['data'] = df['data'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))

In [44]:
df.set_index('data', drop=True)

,nvt
data,
2010-10-09,37.766473
2010-10-11,34.811476
2010-10-14,34.519570
2010-10-16,28.967328
2010-10-17,18.382524
...,...
2021-11-09,3.144203
2021-11-11,4.839684
2021-11-14,6.037593


In [48]:
import json
data_path = '../../Data_project'

with open(os.path.join(data_path, 'nvt.json')) as json_file:
    df_json = json.load(json_file)

In [58]:
df_json = pd.DataFrame(df_json['values'])

In [60]:
import time

df_json['data'] = df_json['x'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(x)))

In [62]:
df_json.set_index('data', drop=True, inplace=True)

In [82]:
df_json.drop(columns='x',inplace=True)
df_json.columns = ['nvt']